In [1]:
import parselmouth as pm
import numpy as np
import plotly.express as px
import pandas as pd
import fasttrackpy.tracking

## First pass

In [2]:
sound = pm.Sound("assets/ay.wav")

In [3]:
print([x for x in dir(sound) if "__" not in x])

['FileFormat', 'ToHarmonicityMethod', 'ToPitchMethod', 'add', 'as_array', 'at_xy', 'autocorrelate', 'centre_time', 'class_name', 'combine_to_stereo', 'concatenate', 'convert_to_mono', 'convert_to_stereo', 'convolve', 'copy', 'cross_correlate', 'de_emphasize', 'deepen_band_modulation', 'divide', 'dt', 'duration', 'dx', 'dy', 'end_time', 'extract_all_channels', 'extract_channel', 'extract_left_channel', 'extract_part', 'extract_part_for_overlap', 'extract_right_channel', 'formula', 'frame_number_to_time', 'full_name', 'get_column_distance', 'get_end_time', 'get_energy', 'get_energy_in_air', 'get_frame_number_from_time', 'get_highest_x', 'get_highest_y', 'get_index_from_time', 'get_intensity', 'get_lowest_x', 'get_lowest_y', 'get_maximum', 'get_minimum', 'get_nearest_zero_crossing', 'get_number_of_channels', 'get_number_of_columns', 'get_number_of_frames', 'get_number_of_rows', 'get_number_of_samples', 'get_power', 'get_power_in_air', 'get_rms', 'get_root_mean_square', 'get_row_distance',

In [4]:
formants = sound.to_formant_burg(time_step = 0.001,
                                 max_number_of_formants = 6, 
                                 maximum_formant = 5000,
                                 window_length = 0.025,
                                 pre_emphasis_from = 50)

In [5]:
print([x for x in dir(formants) if "__" not in x])

['FileFormat', 'centre_time', 'class_name', 'copy', 'dt', 'duration', 'dx', 'end_time', 'frame_number_to_time', 'full_name', 'get_bandwidth_at_time', 'get_end_time', 'get_frame_number_from_time', 'get_number_of_frames', 'get_start_time', 'get_time_from_frame_number', 'get_time_step', 'get_total_duration', 'get_value_at_time', 'info', 'n_frames', 'name', 'nt', 'nx', 'read', 'save', 'save_as_binary_file', 'save_as_short_text_file', 'save_as_text_file', 'scale_times_by', 'scale_times_to', 'scale_x_by', 'scale_x_to', 'shift_times_by', 'shift_times_to', 'shift_x_by', 'shift_x_to', 'start_time', 't1', 't_bins', 't_grid', 'time_range', 'time_step', 'time_to_frame_number', 'tmax', 'tmin', 'total_duration', 'trange', 'ts', 'x1', 'x_bins', 'x_grid', 'xmax', 'xmin', 'xrange', 'xs']


In [6]:
time_domain = formants.xs()

In [7]:
f1 = np.array([formants.get_value_at_time(1, x) for x in time_domain])
f2 = np.array([formants.get_value_at_time(2, x) for x in time_domain])

In [8]:
tracks = np.array(
        [
            [formants.get_value_at_time(i+1, x) 
                for x in time_domain] 
            for i in range(6)
        ]
    )
tracks.shape

(6, 769)

In [9]:
formant_data = pd.DataFrame(
    {"f1": f1,
     "f2": f2,
     "t": time_domain
    }
)

In [10]:
track_plot = px.line(formant_data, x = "t", y = ["f1", "f2"])
track_plot.show()

## With fasttrackpy code

In [11]:
tracked = fasttrackpy.tracking.findformants(sound,
                                 time_step = 0.001,
                                 n_formants = 6, 
                                 maximum_formant = 5000,
                                 window_length = 0.025,
                                 pre_emphasis_from = 50)
tracked.shape

(6, 769)

In [12]:
formant_df = pd.DataFrame({"f1" : tracked[0], "f2" : tracked[1], "t" : time_domain})
formant_df

,f1,f2,t
0,567.192389,1164.183902,0.025406
1,573.631406,1167.431455,0.026406
2,581.554070,1172.692369,0.027406
3,591.472499,1180.737646,0.028406
4,603.382443,1191.824915,0.029406
...,...,...,...
764,274.864856,1051.248536,0.789406
765,291.363797,1055.360784,0.790406
766,310.170734,1066.354259,0.791406
767,328.256330,1090.826708,0.792406


In [13]:
track_plot2 = px.line(formant_df, x = "t", y = ["f1", "f2"])
track_plot2.show()